# Basics of Pinocchio

Welcome in Pinocchio, a C++ library with Python API to efficiently compute all the elements of the model of a rigid robot, and their derivatives. Pinocchio is nicely tailored for generating the motion of a robot using a optimization program. 

Let's start by loading and display the robot model.

In [1]:
import pinocchio
import numpy as np
from numpy.linalg import norm,inv,pinv,svd,eig

You can always alias the long pinocchio namespace to something shorter like pino. In the tuto, we keep the long name for clarity, feel free to shorten it.

## Loading the robot

Pinocchio offers several possibilities to load the model of the robot as a kinematic tree of joints, masses, geometry object and other informative tree "decoration": the most classical is to parse the information from a URDF model. Here we will work with the Talos models: a fixed arm and a pair of floating leg. The collection of Talos models can be downloaded in Ubuntu with the APT package *robotpkg-talos-data*.

In [4]:
urdfFile = '/opt/openrobots/share/talos_data/robots/talos_left_arm.urdf'
rmodel = pinocchio.buildModelFromUrdf(urdfFile)
print(rmodel)

Nb joints = 8 (nq=7,nv=7)
  Joint universe: parent=0
  Joint arm_left_2_joint: parent=0
  Joint arm_left_3_joint: parent=1
  Joint arm_left_4_joint: parent=2
  Joint arm_left_5_joint: parent=3
  Joint arm_left_6_joint: parent=4
  Joint arm_left_7_joint: parent=5
  Joint gripper_left_joint: parent=6



This model as 7 joints, with a configuration space of dimension nq=7. Let's browse quickly the content of the kinematic tree.
In Pinocchio, we do not store explicitly bodies, but the placement of the joint frame, needed to compute the forward kinematics. We name "universe" the first joint frame, and then name each other frame from the name of the joint. Joint information are then stored with the same numbers, 0 being the universe, 1 the first joint, etc until 8 the last joint.

In [5]:
for i,j in enumerate(rmodel.names): print(i,j)

(0, 'universe')
(1, 'arm_left_2_joint')
(2, 'arm_left_3_joint')
(3, 'arm_left_4_joint')
(4, 'arm_left_5_joint')
(5, 'arm_left_6_joint')
(6, 'arm_left_7_joint')
(7, 'gripper_left_joint')


We do our best to keep the names of the URDF file, but as we do not have exactly the same convention for representing the kinematic tree, sometime information are not stored.
Contrary to URDF, our tree is made only of joint. The kinematic order is stored in the parent map rmodel.parent. The type of the joints (revolute X, free flyer, spherical, prismatic, etc) are stored in the rmodel.joints map. Each joint is placed (ie position and orientation) with respect to its parent, and the placement is stored in rmodel.placement.

In [11]:
for i,(j,p,M) in enumerate(zip(rmodel.joints,rmodel.parents,rmodel.jointPlacements)): 
    print(i,j,"parent=%d"%p,M)

(0, JointModelRX
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1
, 'parent=0',   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0
)
(1, JointModelRX
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1
, 'parent=0',   R =
1 0 0
0 1 0
0 0 1
  p = 0.00493  0.1365 0.04673
)
(2, JointModelRZ
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1
, 'parent=1',   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0
)
(3, JointModelRY
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1
, 'parent=2',   R =
1 0 0
0 1 0
0 0 1
  p =   0.02      0 -0.273
)
(4, JointModelRZ
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1
, 'parent=3',   R =
1 0 0
0 1 0
0 0 1
  p =   -0.02       0 -0.2643
)
(5, JointModelRX
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1
, 'parent=4',   R =
1 0 0
0 1 0
0 0 1
  p = 0 0 0
)
(6, JointModelRY
  index: 18446744073709551615
  index q: -1
  index v: -1
  nq: 1
  nv: 1


Masses and inertias are also stored, along with other informations we will discuss later. The dimension of the configuration space is denoted rmodel.nq, while the number of degrees of freedom is rmodel.nv (sometime, nq>nv, here both are equals). The number of joints is given by rmodel.njoints: here as all joints have dimension 1 and we must account for the universe, we have njoints=nq+1.

In [18]:
print(rmodel.nq,rmodel.nv,rmodel.njoints)

(7, 7, 8)


## The convenient RobotWrapper and the display
Most of the Python API simply copy the C++ API. We are using Boost::Python to simply copy the API from C++ to Python, with minor re-arragement.
Only a helper object has been introduced in Python to make the life of the newcommer easier: the RobotWrapper. It is in particular helpful to load completely the model and display it in Gepetto-Viewer. RobotWrapper loads the URDF model but also loads the geometry models for the display and the collision checker. For that, it needs some extra path information that are gathered in the robots.py. Go and have a look at it.

In [21]:
import robots
robot = robots.loadTalosArm()
print(robot.model)

Nb joints = 8 (nq=7,nv=7)
  Joint universe: parent=0
  Joint arm_left_2_joint: parent=0
  Joint arm_left_3_joint: parent=1
  Joint arm_left_4_joint: parent=2
  Joint arm_left_5_joint: parent=3
  Joint arm_left_6_joint: parent=4
  Joint arm_left_7_joint: parent=5
  Joint gripper_left_joint: parent=6



Finally, it loaded the same robot model than before, but also did other interresting parsing, that we will discover later. Using the extra info we can initialize the display. Start gepetto-gui from a terminal (it has been installed from the 2 packages robotpkg-gepetto-viewer-corba and robotpkg-osg-dae). Gepetto-gui starts a graphic server, and we will open a client to this server in Python using the method implemented in RobotWrapper:

In [26]:
robot.initDisplay(loadModel=True)

You can see in the GUI window that a blue world has been loaded, with all the meshes of the robot stacked in the middle of the scene. Gepetto-viewer is indeed a rigid-object viewer, that display each mesh at a given placement (gepetto-viewer has no idea of the kinematic chain). You then need pinocchio to compute the placement of all the bodies and place them at the right position and orientation. This is all done in RobotWrapper. 

In [27]:
robot.display(robot.q0)

where robot.q0 is a configuration of the robot. RobotWrapper was designed initially as a reference example where a newcommer can pick basic ways of using Pinocchio. Don't hesitate to go inside this Python file and have a look at how it is done. 
The robot configuration can be changed in Gepetto-viewer with:

In [29]:
from pinocchio.utils import rand,zero,eye
robot.display(rand(robot.model.nq)*2-1)

## Spatial algebra
The scene, and later the movement and forces in the scene are modeled following Featherstone's Spatial Algebra. Placement, i.e. rotation and translation of frames (and bodies) are stored in objects of the class SE3. Rigid velocities and acceleration are stored in the class Motion, forces in the class Forces and masses/inertias in the class Inertias.

In [30]:
from pinocchio import SE3,Motion,Force,Inertia
M = SE3.Random()
nu = Motion.Random()
phi = Force.Random()
Y = Inertia.Random()
print(M,nu,phi,Y)


(  R =
 0.444451 -0.414554  0.794109
 0.175408 -0.829041 -0.530964
 0.878462  0.375281 -0.295751
  p =  0.257742 -0.270431 0.0268018
,   v = 0.904459  0.83239 0.271423
  w =  0.434594 -0.716795  0.213938
,   f = -0.967399 -0.514226 -0.725537
tau =  0.608354 -0.686642 -0.198111
,   m = 1.04857
  c =   0.22528 -0.407937  0.275105
  I = 
  1.47786 -0.215983 0.0148097
-0.215983   1.60849  0.484213
0.0148097  0.484213  0.778174)


These objects store linear and angular part asside, but we often have to come back to a plain vector/matrix representation. In that case, contrary to Featherstone, we rather store linear part first and angular second.

In [32]:
print(nu,nu.vector.T)

(  v = 0.904459  0.83239 0.271423
  w =  0.434594 -0.716795  0.213938
, matrix([[ 0.90445945,  0.83239014,  0.27142346,  0.43459386, -0.71679489,
          0.21393775]]))


# Forward kinematics: the first algorithm
## Model and data
Before calling the algorithm, let's introduce a specificity of Pinocchio: the strict separation between constant model element in the Model class, and all the buffers for storing algorithm quantities in the Data class. 

In [34]:
rdata = rmodel.createData()

The RobotWrapper creates a Data object by default:

In [35]:
print(robot.data)

The idea is that the same model can be used by different part of the algorithm to compute different values from different argument. For example, in a optimal-control implementation of Pinocchio, you likely want to have a single robot model for all your problem, but several data for each node of your optimal control solver. In the tuto, we will for example use the fact that the numerical algorithm has its own Data, while the RobotWrapper use another Data for computing body placements when displaying something.

## Calling the algorithm implementation
The forward kinematics simply compute the placement of every joint frame for a given configuration q. 

In [39]:
q = rand(rmodel.nq)
pinocchio.forwardKinematics(rmodel,rdata,q)
for i,M in enumerate(rdata.oMi[1:]): print(i,M)

(0,   R =
        1         0         0
        0  0.910158 -0.414261
        0  0.414261  0.910158
  p = 0.00493  0.1365 0.04673
)
(1,   R =
  0.57594 -0.817492         0
 0.744047  0.524196 -0.414261
 0.338655   0.23859  0.910158
  p = 0.00493  0.1365 0.04673
)
(2,   R =
 0.333886 -0.817492  0.469284
 0.768887  0.524196  0.366103
-0.545283   0.23859  0.803581
  p = 0.0164488  0.264474  -0.19497
)
(3,   R =
 0.234284 -0.851401  0.469284
 0.825776  0.429025  0.366103
-0.513035  0.301751  0.803581
  p = -0.114261  0.152336 -0.396451
)
(4,   R =
 0.234284 -0.628908  0.741341
 0.825776  0.531161  0.189636
-0.513035  0.567753   0.64378
  p = -0.114261  0.152336 -0.396451
)
(5,   R =
 -0.23867 -0.628908   0.73994
 0.563092  0.531161  0.633084
-0.791179  0.567753  0.227361
  p = -0.114261  0.152336 -0.396451
)
(6,   R =
  -0.23867  -0.321799   0.916232
  0.563092   0.722831   0.400553
 -0.791179   0.611523 0.00868463
  p =  -0.21762 0.0855547   -0.4128
)


When calling forwardKinematics, the model rmodel has not been changed, while the results of the algorithm have been stored in the robot Data. Some algorithm have a main result that is returned by the algorithm (like for example crba that returns the robot mass matrix), but in general the main algorithm results are just several changes in the tables stored in robot Data.

## Computing the end-effector position
In a first time, we want to control the end effector position (3D). It is given by:

In [41]:
print("End effector = " , rdata.oMi[-1].translation.T)

('End effector = ', matrix([[-0.21762018,  0.08555466, -0.41279973]]))


# Optimizing the end effector position
We can now set up the first optimization problem that compute a robot configuration minimizing the distance between the position of the end-effector and a 3D target. 
For that we will use the fmin_slsqp from SciPy. You need to define the cost function cost(q) that returns a scalar measuring this distance. Beware of a painful implementation detail!
## The classes Matrix and Array of NumPy
NumPy implements matrices and vectors with the class np.array, represented as tensors i.e N-D tables, where vectors would be N=1, matrices N=2. Tensor product, that boils down to Matrix-Vector and Matrix-Matrix products in the corresponding dimension, is obtained with the np.dot operator : np.dot(A,x) to multiply $A*x$. The multiplication operator is not the matrix product but the coefficient-wise product.

On the other hand, NumPy also introduces the class Matrix as a specific implementation of the 2-D array, and overload the __ mult __ operator to match the matrix operator, hence $A*x$ is directly obtain with the \* symbol.

Pinocchio has been implemented with the Matrix class. Any other Python package is rather implemented with the Array class. In particular, the SciPy optimizers are with Array. So, we will painfully have to convert array to matrix before calling Pinocchio algorithms, and back to array when returning the results to the optimizer. 


In [42]:
m2a = lambda m: np.array(m.flat)
a2m = lambda a: np.matrix(a).T

## Cost function
That said, the cost function simply has to call forwardKinematics, and return the difference between the computed effector position and a reference.

In [44]:
ref = np.matrix([.3,.3,.3]).T
def cost(x):
    q = a2m(x)
    pinocchio.forwardKinematics(rmodel,rdata,q)
    M = rdata.oMi[-1]
    p = M.translation
    residuals = m2a(p-ref)
    return sum( residuals**2)
x0 = np.random.rand(rmodel.nq)
print(cost(x0))

0.9821014346907664


## FMin
The optimizer chosen for the class is SLSQP which is a SQP accepting equality, inequality and bound constraints, using BFGS for quasi-newton acceleration and a least-square QP for computing the Newton step. It is quite a good solver, although not strong enough for implementing real robotics application. It is yet quite comfortable for a class to have access to it through the easy package SciPy.
The API of slsqp is as follows:

In [48]:
from scipy.optimize import fmin_slsqp
help(fmin_slsqp)

Help on function fmin_slsqp in module scipy.optimize.slsqp:

fmin_slsqp(func, x0, eqcons=(), f_eqcons=None, ieqcons=(), f_ieqcons=None, bounds=(), fprime=None, fprime_eqcons=None, fprime_ieqcons=None, args=(), iter=100, acc=1e-06, iprint=1, disp=None, full_output=0, epsilon=1.4901161193847656e-08, callback=None)
    Minimize a function using Sequential Least SQuares Programming
    
    Python interface function for the SLSQP Optimization subroutine
    originally implemented by Dieter Kraft.
    
    Parameters
    ----------
    func : callable f(x,*args)
        Objective function.
    x0 : 1-D ndarray of float
        Initial guess for the independent variable(s).
    eqcons : list, optional
        A list of functions of length n such that
        eqcons[j](x,*args) == 0.0 in a successfully optimized
        problem.
    f_eqcons : callable f(x,*args), optional
        Returns a 1-D array in which each element must equal 0.0 in a
        successfully optimized problem.  If f_eqcon

Here we only use the initial guess and the cost function.

In [50]:
result = fmin_slsqp(x0=np.zeros(rmodel.nq),
                    func=cost)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.8019119356477975e-07
            Iterations: 12
            Function evaluations: 108
            Gradient evaluations: 12


In [51]:
qopt = a2m(result)
robot.display(qopt)

## Using the viewer to interpret the solver
Let's use the viewer to see what the solver is doing.
First, let's add a visual object to mark the target.

In [64]:
gview = robot.viewer.gui
gview.addSphere('world/target',0.1,[1.,0.,0.,1.]) # radius, [R,G,B,A]
gview.applyConfiguration('world/target',[.3,.3,.3,0.,0.,0.,1.]) # x,y,z,quaternion
gview.refresh()

Gepetto-viewer accepts many types of simple 3d geom (see the /opt/openrobots/share/idl/gepetto/corbaserver/graphical-interface.idl API for a list), and wait for the "refresh" order before placing all of them.

We can also change the robot configuration while the solver works, to render the current guess of the algorithm. For that, slsqp offers the callback interface.

In [67]:
def callbackDisp(x):
    import time
    q = a2m(x)
    robot.display(q)
    time.sleep(.1)
result = fmin_slsqp(x0=np.zeros(rmodel.nq),
                    func=cost,
                    callback=callbackDisp)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 1.8019119356477975e-07
            Iterations: 12
            Function evaluations: 108
            Gradient evaluations: 12


# The complete program
In general, it is recommanded to store all interesting information and data related to the optimization program inside a dedicated object, whose paramaters are initialized in the constructer. Then the cost (and later constraint, callback, etc) functions are object methods. A complete implementation of the 3D example is given in arm3d.py

In [ ]:
# %load arm3d.py
from robots import loadTalosArm
from scipy.optimize import fmin_slsqp
import pinocchio
from pinocchio.utils import *
from numpy.linalg import norm,inv,pinv,eig,svd

m2a = lambda m: np.array(m.flat)
a2m = lambda a: np.matrix(a).T

robot   = loadTalosArm()
robot.initDisplay(loadModel=True)

class OptimProblem:
    def __init__(self,rmodel,rdata,gview=None):
        self.rmodel = rmodel
        self.rdata = rdata
        self.ref = [ .3, 0.3, 0.3 ]     # Target position
        self.idEff = -1                 # ID of the robot object to control

    def cost3(self,x):
        q = a2m(x)
        pinocchio.forwardKinematics(self.rmodel,self.rdata,q)
        M = self.rdata.oMi[self.idEff]
        self.residuals = m2a(M.translation) - self.ref
        return sum( self.residuals**2 )

    def initDisplay(self,gview=None):
        self.gview = gview
        if gview is None: return
        self.gobj = "world/target3d"
        self.gview.addSphere(self.gobj,.03,[1,0,0,1])
        self.gview.applyConfiguration(self.gobj,self.ref+[0,0,0,1])
        self.gview.refresh()

    def callback(self,x):
        import time
        q = a2m(x)
        robot.display(q)
        time.sleep(1e-2)

pbm = OptimProblem(robot.model,robot.model.createData(),robot.viewer.gui)

x0  = m2a(robot.q0)
result = fmin_slsqp(x0=x0,
                    func=pbm.cost3,
                    callback=pbm.callback)
qopt = a2m(result)


# Optimization of the effector placement
The forward kinematics indeed computes the placement of the last frame, i.e the rotation R and the translation p, denoted M = \[R,p\] $\in SE(3)$.
We need to define a metric to score the distance between to frames $M_1$ and $M_2$. Several metrics can be chosen, but a nice one is given by the SE3 logarithm function, that converts the gap between two frames into the velocity that should applied (constant) during t=1 to bridge the gap a displace $M_1$ into $M_2$.  

In [74]:
M1 = SE3.Random()
M2 = SE3.Random()
nu = pinocchio.log(M1.inverse()*M2)
print(nu,nu.vector.T)

(  v =  0.565222 -0.815087  0.418248
  w =  0.499815  0.564272 -0.207136
, matrix([[ 0.5652219 , -0.81508665,  0.41824769,  0.4998151 ,  0.56427201,
         -0.20713564]]))


The norm of the logarithm is a proper cost function: it is 0 if and only if the two frames matches, and positive otherwise ; it is smooth; compare to other fancy metrics, it is easy to differenciate (at least, there are some well founded rules to differentiate the logarithm and related operators).

Modify the program above to search for the robot configuration so that the end effector is placed at a reference position and orientation (solution only if need be). 

In [75]:
# %load arm6d.py

# Frames

We already said that the kinematic tree is composed of a hierarchy of frames corresponding to the output of each joint. In practice, we find it useful to attach additional frames to these main frames. We name the main frames defining the kinematic tree by Joint Frames, stored in rdata.oMi. The other frames are described in the model by the rmodel.frames list, each object storing its name, the index of its parent joint frame and the fixed placement with respect to its parent.


In [77]:
for i,f in enumerate(rmodel.frames): print(i,f.name,f.parent)

(0, 'universe', 0, pinocchio.libpinocchio_pywrap.FrameType.FIXED_JOINT)
(1, 'root_joint', 0, pinocchio.libpinocchio_pywrap.FrameType.FIXED_JOINT)
(2, 'arm_left_1_link', 0, pinocchio.libpinocchio_pywrap.FrameType.BODY)
(3, 'arm_left_2_joint', 1, pinocchio.libpinocchio_pywrap.FrameType.JOINT)
(4, 'arm_left_2_link', 1, pinocchio.libpinocchio_pywrap.FrameType.BODY)
(5, 'arm_left_3_joint', 2, pinocchio.libpinocchio_pywrap.FrameType.JOINT)
(6, 'arm_left_3_link', 2, pinocchio.libpinocchio_pywrap.FrameType.BODY)
(7, 'arm_left_4_joint', 3, pinocchio.libpinocchio_pywrap.FrameType.JOINT)
(8, 'arm_left_4_link', 3, pinocchio.libpinocchio_pywrap.FrameType.BODY)
(9, 'arm_left_5_joint', 4, pinocchio.libpinocchio_pywrap.FrameType.JOINT)
(10, 'arm_left_5_link', 4, pinocchio.libpinocchio_pywrap.FrameType.BODY)
(11, 'arm_left_6_joint', 5, pinocchio.libpinocchio_pywrap.FrameType.JOINT)
(12, 'arm_left_6_link', 5, pinocchio.libpinocchio_pywrap.FrameType.BODY)
(13, 'arm_left_7_joint', 6, pinocchio.libpinocchi

For convenience, we also describe if this frame was parsed as a body frame, a joint frame (yes, joint frames are copied again in the rmodel.frames list as it makes the access to frame more generic) or as fixed joints (that is a pretty classical trick in URDF. 

For example, the joint frame attached to the foot of a biped robot is often located at its ankle, i.e. 5 to 10 cm above the ground. We then also attach a second frame on the foot sole, to make it easier to write the contact constraints. And similarly for a quadruped, the last joint frame is at the knew, and we rather attach another frame at the tip of the leg.

Frames are best indexed by their name:

In [79]:
fid = rmodel.getFrameId('gripper_left_fingertip_2_link')
print(fid)

26


The joint placement are stored in rdata.oMi. The frame placements are stored in rdata.oMf. By default, the forwardKinematics does not reevaluate the oMf. Do it with:

In [80]:
pinocchio.updateFramePlacements(rmodel,rdata)

Note that this method does not need q to evaluate the oMf from the oMi.

Modify the above example to optimize the placement of the robot effector tip rather than its wrist.

# With joint limits
The joint limits are also parsed from the URDF model and stored in rmodel.lowerLimit and rmodel.upperLimit.

In [82]:
print(rmodel.lowerPositionLimit.T,rmodel.upperPositionLimit.T)

(matrix([[ 0.        , -2.44346095, -2.35619449, -2.53072742, -1.3962634 ,
         -0.6981317 , -1.04719755]]), matrix([[2.87979327, 2.44346095, 0.        , 2.53072742, 1.3962634 ,
         0.6981317 , 0.        ]]))


fmin_slsqp accepts bound constraints as a list of 2-ples (lower,upper).

In [85]:
bounds=[ (l,u) for l,u in zip(robot.model.lowerPositionLimit.flat,robot.model.upperPositionLimit.flat) ]